<a href="https://colab.research.google.com/github/ingabLee/Transformers_book/blob/main/Transformers_Chap_37_42.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install lib
# datasets==1.4.1
!pip install datasets
!pip install transformers
!pip install soundfile
!pip install jiwer

In [ ]:
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_name = "elgeish/wav2vec2-base-timit-asr"

# model, processor load
process = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# model eval change.
model.eval()

In [ ]:
dataset = load_dataset("timit_asr")

In [ ]:
import soundfile as sf
import IPython.display as ipd
import numpy as np
import random

# dataset['test']['files']에 있는 첫번째 데이터를 df.read에 투입
speech,_ = sf.read(dataset['test']['file'][0])
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

In [ ]:
# processor sound data
inputs = process(speech, sampling_rate=16000, return_tensors="pt", padding="longest")

# 음성 데이터 처리 결과의 input_values차원 확인
inputs['input_values'].shape

In [ ]:
# no gradient calc. 이는 모델 eval모드에 대한 필요한 조치
with torch.no_grad():

  # model inputs.input_values
  outputs = model(inputs.input_values)

  # print logit(outputs.logits) shape
  print(outputs.logits.shape)

  # output.logits의 마지막 축 기준으로 argmax적용
  # 가장 큰 인덱스를 predicted_ids로 적용
  predicted_ids = outputs.logits.argmax(-1)

  print(predicted_ids)

  # predicted_ids의 첫번째 데이터를 processor.tokenizer.batch_decode로 디코딩
  # predicted_ids[0]은 입력문장 중 첫번째라는 뜻이나
  # 입력무장을 하나만 넣었으므로 predicted_ids[0]은 입력문장 그자체임. ??
  # 디코딩 결과 출력
  print(f"predicted:{process.tokenizer.batch_decode(predicted_ids)[0]}")

  # dataset['test']['text'][0] => 정답 출력
  print(f"answer:{dataset['test']['text'][0]}")
